# Special I/O


In [1]:
import h5rdmtoolbox as h5tbx

## Creating datasets and CSV file(s)
Datasets can be created directly form a single or from multiple files. Let's first create two simple CSV files:

In [2]:
import pandas as pd
import numpy as np

# first
df = pd.DataFrame({'x': np.random.random((4, )),
                   'y': np.random.random((4, ))})
csv_filename1 = h5tbx.utils.generate_temporary_filename(suffix='.csv')
df.to_csv(csv_filename1, index=None)

# second
df = pd.DataFrame({'x': np.random.random((8, )),
                   'y': np.random.random((8, ))})
csv_filename2 = h5tbx.utils.generate_temporary_filename(suffix='.csv')
df.to_csv(csv_filename2, index=None)

Create from a single file:

In [3]:
with h5tbx.H5File() as h5:
    h5.create_dataset_from_csv(csv_filename=csv_filename1)
    h5.dump()

For creating from multiple CSV files, it must be decided whether to stack (datasets must have same size) or concatenate them:

... concatenating:

In [4]:
with h5tbx.H5File() as h5:
    h5.create_datasets_from_csv(csv_filenames=[csv_filename1, csv_filename2], combine_opt='concatenate')
    h5.dump()

... stacking:

In [5]:
with h5tbx.H5File() as h5:
    h5.create_datasets_from_csv(csv_filenames=[csv_filename2, csv_filename2], combine_opt='stack')
    h5.dump()

## Creating datasets and image file(s)
A dataset can be created from image data. The data can be provided as a list of numpy arrays:

In [6]:
with h5tbx.H5File() as h5:
    h5.create_dataset_from_image([np.random.random((20, 10))] * 5,
                                 'testimg', axis=0)
    h5.dump()

... or as a iterable object which provides the image data one at a time:

In [7]:
class ImgReader:
    """Dummy Image Reader"""
    def __init__(self, imgdir):
        self._imgdir = imgdir
        self._index = 0
        self._size = 5

    def read_img(self):
        # provide random image. Use case would read from file...
        return np.random.random((20, 10))

    def __iter__(self):
        return self

    def __len__(self):
        return self._size

    def __next__(self):
        if self._index < self._size:
            self._index += 1
            return self.read_img()
        raise StopIteration

In [8]:
imgreader = ImgReader('testdir')
with h5tbx.H5File() as h5:
    h5.create_dataset_from_image(imgreader, 'testimg', axis=0)
    h5.dump()